In [22]:
!wget https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/vocab.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
--2023-09-06 07:16:47--  https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/vocab.txt
Resolving deepchemdata.s3-us-west-1.amazonaws.com (deepchemdata.s3-us-west-1.amazonaws.com)... 3.5.160.160, 52.219.117.146, 3.5.162.13, ...
Connecting to deepchemdata.s3-us-west-1.amazonaws.com (deepchemdata.s3-us-west-1.amazonaws.com)|3.5.160.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3524 (3.4K) [text/plain]
Saving to: ‘vocab.txt’

vocab.txt           100%[===================>]   3.44K  --.-KB/s    in 0s      

2023-09-06 07:16:48 (12.9 MB/s) - ‘vocab.txt’ saved [3524/3524]



In [198]:
from deepchem.feat.smiles_tokenizer import SmilesTokenizer


In [7]:

from transformers import AutoModelForMaskedLM, AutoModel, AutoTokenizer, pipeline, RobertaModel, RobertaTokenizer
from transformers import AutoModelForSequenceClassification

# model = AutoModelForMaskedLM.from_pretrained("DeepChem/ChemBERTa-77M-MTR")
tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MTR")
model = AutoModel.from_pretrained("DeepChem/ChemBERTa-77M-MTR")
model = AutoModelForSequenceClassification.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k", num_labels=2, problem_type="multi_label_classification")
# tokenizer = AutoTokenizer.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)


Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MTR and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
model

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(600, 384, padding_idx=1)
    (position_embeddings): Embedding(515, 384, padding_idx=1)
    (token_type_embeddings): Embedding(1, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.144, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-2): 3 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.109, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropou

In [146]:
vocab = '/root/Competitions/DACON/4. JUMP AI 2023/example/vocab.txt'

# Pt로 데려온것이랑 차이 
# DC tokenizer -> input_ids, token_type_ids, attention_mask 를 제공
# Hugging Face -> input_ids, x , attention_mask
## tokenizer.encoder(smiles) -> input_ids를 리턴 
deep_chem_tokenizer = SmilesTokenizer(vocab_file=vocab, max_len=model.config.max_position_embeddings)
deep_chem_tokenizer

SmilesTokenizer(name_or_path='', vocab_size=591, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [4]:
smiles1 = 'CC(C)Nc1ccnc(N2CCN(Cc3cccs3)C(CCO)C2)n1'
smiles2 = 'C1=Cn2nc(/C=C/c3cccs3)nc2-c2ccccc2O1'
smiles3 = 'CCN1CCN(C(=O)c2cc3c(=O)n4cc(C)ccc4nc3n2C)CC1'

batch = [smiles1]
inputs = tokenizer.encode_plus(smiles1, padding=True, truncation=True, return_tensors='pt', add_special_tokens=True, max_length=model.config.max_position_embeddings)
print(inputs)
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
print('input_ids', input_ids.shape)
print('attention_mask', attention_mask.shape)
ret = model(input_ids)
print(ret)
attention = ret[-1]
input_id_list = input_ids[0].tolist() # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)# head_view(attention, tokens)

ret.logits.shape
# print(ret.pooler_output.shape, ret.last_hidden_state.shape)

{'input_ids': tensor([[12, 16, 16, 17, 16, 18, 23, 15, 20, 15, 15, 25, 15, 17, 23, 21, 16, 16,
         23, 17, 16, 15, 26, 15, 15, 15, 42, 26, 18, 16, 17, 16, 16, 19, 18, 16,
         21, 18, 25, 20, 13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
input_ids torch.Size([1, 41])
attention_mask torch.Size([1, 41])
SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0923, -0.0815]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


torch.Size([1, 2])

In [195]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_inputs = tokenizer(batch_sentences)
print(encoded_inputs)

{'input_ids': [[101, 1252, 1184, 1164, 1248, 6462, 136, 102], [101, 1790, 112, 189, 1341, 1119, 3520, 1164, 1248, 6462, 117, 21902, 1643, 119, 102], [101, 1327, 1164, 5450, 23434, 136, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}


In [76]:
from deepchem.molnet import load_bbbp, load_clearance, load_clintox, load_delaney, load_hiv, load_qm7, load_tox21

In [158]:
import numpy as np
import pandas as pd
tasks, (train_df, valid_df, test_df), transformers = load_clearance()

def proc(data):
    y = data.y
    ids = data.ids 
    df = np.concatenate([ids.reshape(-1, 1), y], axis=1)
    df = pd.DataFrame(df, columns=['text', 'labels'])
    
    return df

In [159]:
train_df = proc(train_df)
train_df

,text,labels
0,CS(=O)(=O)c1ccc(cc1)[C@@H](O)[C@@H](CF)NC(=O)C...,4.251063
1,CC(C)CCOc1ccc(N)cc1,5.01728
2,COc1cc(cc(OC)c1O)C(=O)OCCCCNC(=N)N,4.934762
3,C=CC(=O)Nc1ccccc1,3.507358
4,C[C@@H](C(=O)NS(=O)(=O)C)c1ccc(OS(=O)(=O)C(F)(...,1.386294
...,...,...
664,C#Cc1cccc(Nc2ncnc3cc4OCCOCCOCCOc4cc23)c1,3.130263
665,O=C(N1CCOCC1)N2CCn3cc(C4=C(C(=O)NC4=O)c5cnc6cc...,3.663562
666,CCOc1nc2cccc(C(=O)O)c2n1Cc3ccc(cc3)c4ccccc4c5n...,1.386294
667,NC(=O)NC(=O)C(Nc1ccc2CCCc2c1)c3ccccc3,4.024458


In [160]:
valid_df = proc(valid_df)
valid_df

,text,labels
0,O[C@@H](CNCCOc1cccc(CNCCc2ccccc2F)c1)c3ccc(O)c...,3.178054
1,Clc1ccc(CN2CCNCC2)cc1C(=O)NCC34CC5CC(CC(C5)C3)C4,1.609438
2,COCCN(C)c1ccc(Nc2ncc3cc(ccc3n2)c4ccncc4)cc1,3.73146
3,C[C@@H](N(CC1CCS(=O)(=O)CC1)C(=O)Cc2ccc(c(F)c2...,2.089392
4,CC(C)N1CCN(Cc2oc(nc2)c3cc(cc4[nH]ncc34)c5cccc6...,2.781301
...,...,...
79,O[C@@H](CNCCSCCCOCCc1ccccc1)c2ccc(O)c3NC(=O)Sc23,5.01728
80,Cc1ccccc1c2c(C(=O)O)n(CCCOc3cccc4ccccc34)c5ccc...,4.820523
81,Clc1ccc2ncc(c3cccc(NC4CNC4)n3)n2c1,2.998229
82,OCc1cc(ccc1O)C(O)CNCCCCCCOCCCCc2ccccc2,4.454347


In [168]:
from datasets

TypeError: Module.train() got an unexpected keyword argument 'train_dataset'